# 멀티 에이전트 시뮬레이션으로 채팅 봇 평가하기

고객 지원 도우미와 같은 채팅 봇을 구축할 때는 봇의 성능을 제대로 평가하기가 어려울 수 있습니다. 코드를 변경할 때마다 수동으로 봇과 집중적으로 상호작용해야 하므로 시간이 많이 소요됩니다.

평가 프로세스를 더 쉽고 재현 가능하게 만드는 한 가지 방법은 사용자 상호작용을 시뮬레이션하는 것입니다.

LangGraph를 사용하면 이를 쉽게 설정할 수 있습니다. 다음은 대화를 시뮬레이션하기 위해 '가상 사용자'를 만드는 방법의 예시입니다.

전체 시뮬레이션은 다음과 같습니다:


<img src="./img/virtual_user_diagram.png" width="50%">

In [ ]:
!pip install -U langgraph langchain langchain_openai

In [ ]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")

## 1. 채팅 봇 정의하기

다음으로 채팅 봇을 정의합니다.  봇의 API가 메시지 목록을 수락하고 메시지로 응답한다고 가정합니다. 이를 업데이트하려면 아래 시뮬레이터에서 이 섹션과 "get_messages_for_agent" 함수만 변경하면 됩니다.

`my_chat_bot` 내의 구현은 구성 가능하며 다른 시스템에서도 실행할 수 있습니다
(예: 시스템이 파이썬으로 실행되지 않는 경우).

In [1]:
from typing import List

import openai


# 테스트하고자 하는 Agent
def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "당신은 항공사의 고객 지원 상담원입니다.",
    }
    messages = [system_message] + messages
    completion = openai.chat.completions.create(
        messages=messages, model="gpt-4o"
    )
    return completion.choices[0].message.model_dump()

In [2]:
my_chat_bot([{"role": "user", "content": "hi!"}])

{'content': '안녕하세요! 어떻게 도와드릴까요? 항공편 예약, 일정 변경, 수하물 정보 등 어떤 도움이 필요하신지 말씀해 주세요.',
 'role': 'assistant',
 'function_call': None,
 'tool_calls': None}

## 2. 시뮬레이션 사용자 정의

이제 시뮬레이션 사용자를 정의하겠습니다.  이것은 우리가 원하는 무엇이든 될 수 있지만, LangChain 봇으로 구축하겠습니다.

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

system_prompt_template = """당신은 항공사의 고객입니다. \
고객 지원 담당자와 대화하고 있습니다. \

{instructions}

대화를 마치면 '완료'""라는 한 단어로 응답하세요.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
instructions = """당신의 이름은 해리슨입니다. 알래스카 여행에 대한 환불을 받으려고 합니다. 모든 돈을 돌려받기를 원합니다. \
이 여행은 5년 전에 이루어졌습니다."""

prompt = prompt.partial(name="Harrison", instructions=instructions)

model = ChatOpenAI(model="gpt-4o")

simulated_user = prompt | model

In [13]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="안녕하세요! 무엇을 도와드릴까요?")]
simulated_user.invoke({"messages": messages})

AIMessage(content='안녕하세요, 저는 해리슨입니다. 5년 전에 알래스카 여행을 다녀왔는데, 그 여행에 대한 환불을 받으려고 합니다. 모든 돈을 돌려받고 싶습니다. 어떻게 해야 하나요?', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 108, 'total_tokens': 159}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'stop', 'logprobs': None}, id='run-9859ebb7-c1d3-449f-a739-ee0a2d5bff9f-0', usage_metadata={'input_tokens': 108, 'output_tokens': 51, 'total_tokens': 159})

## 3. 에이전트 시뮬레이션 정의하기

아래 코드는 시뮬레이션을 실행하기 위한 LangGraph 워크플로우를 생성합니다. 주요 구성 요소는 다음과 같습니다:

1. 두 개의 노드: 하나는 시뮬레이션된 사용자를 위한 것이고 다른 하나는 채팅 봇을 위한 것입니다.
2. 조건부 중지 기준이 있는 그래프 자체.

자세한 내용은 아래 코드의 댓글을 참조하세요.

**노드**

먼저 그래프에서 노드를 정의합니다. 노드는 메시지 목록을 받아 상태에 추가할 메시지 목록을 반환해야 합니다.
이는 위에 있는 채팅 봇과 시뮬레이션된 사용자를 감싸는 wrapper가 될 것입니다.

**참고**: 여기서 한 가지 까다로운 점은 어떤 메시지가 어떤 메시지인지 구분하는 것입니다. 채팅 봇과 시뮬레이션된 사용자는 모두 LLM이므로 둘 다 AI 메시지에 반응할 것입니다. 우리의 상태는 인간과 AI 메시지가 번갈아 나오는 목록이 될 것입니다. 즉, 노드 중 하나에 대해 AI와 인간의 역할을 뒤집는 로직이 있어야 합니다. 이 예제에서는 휴먼 메시지가 시뮬레이션된 사용자가 보낸 메시지라고 가정하겠습니다. 즉, 시뮬레이션된 사용자 노드에 AI와 인간 메시지를 교환하기 위한 로직이 필요합니다.

먼저 채팅 봇 노드를 정의해 보겠습니다.

In [14]:
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage


def chat_bot_node(messages):
    # Convert from LangChain format to the OpenAI format, which our chatbot function expects.
    messages = [convert_message_to_dict(m) for m in messages]
    # Call the chat bot
    chat_bot_response = my_chat_bot(messages)
    # Respond with an AI Message
    return AIMessage(content=chat_bot_response["content"])

다음으로 시뮬레이션된 사용자에 대한 노드를 정의해 보겠습니다. 여기에는 메시지의 역할을 바꾸기 위한 약간의 로직이 필요합니다.

In [15]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(messages):
    # Swap roles of messages
    new_messages = _swap_roles(messages)
    # Call the simulated user
    response = simulated_user.invoke({"messages": new_messages})
    # This response is an AI message - we need to flip this to be a human message
    return HumanMessage(content=response.content)

**Edges**

이제 에지에 대한 로직을 정의해야 합니다. 주요 로직은 시뮬레이션된 사용자가 이동한 후에 발생하며, 두 가지 결과 중 하나로 이어져야 합니다:

- 계속 진행하여 고객 지원 봇에 전화합니다.
- 또는 대화가 끝나고 대화가 종료됩니다.

그렇다면 대화가 종료되는 로직은 무엇일까요? 인간 챗봇이 '완료'로 응답하거나(시스템 프롬프트 참조) 대화가 6개를 초과하는 경우로 정의하겠습니다(이 예제를 짧게 유지하기 위해 임의의 숫자입니다).

In [16]:
def should_continue(messages):
    if len(messages) > 6:
        return "end"
    elif messages[-1].content == "완료":
        return "end"
    else:
        return "continue"

**그래프**

이제 시뮬레이션을 설정하는 그래프를 정의할 수 있습니다!

In [17]:
from langgraph.graph import END, MessageGraph

graph_builder = MessageGraph()
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# Every response from  your chat bot will automatically go to the
# simulated user
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # If the finish criteria are met, we will stop the simulation,
    # otherwise, the virtual user's message will be sent to your chat bot
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# The input will first go to your chat bot
graph_builder.set_entry_point("chat_bot")
simulation = graph_builder.compile()

## 4. 시뮬레이션 실행

이제 채팅 봇을 평가할 수 있습니다! 빈 메시지로 채팅 봇을 호출할 수 있습니다(이렇게 하면 채팅 봇이 초기 대화를 시작하는 시뮬레이션을 할 수 있습니다).

In [18]:
for chunk in simulation.stream([]):
    # Print out all events aside from the final end chunk
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': AIMessage(content='안녕하세요! 항공사 고객 지원팀입니다. 어떻게 도와드릴까요?')}
----
{'user': HumanMessage(content='안녕하세요, 저는 해리슨입니다. 5년 전에 알래스카로 여행을 갔었는데, 그 여행에 대한 환불을 받고 싶습니다. 모든 돈을 돌려받기를 원합니다.', id='37d8f1d4-6400-407e-ae10-28edb512f024')}
----
{'chat_bot': AIMessage(content='안녕하세요, 해리슨님. 고객 지원팀에 문의해 주셔서 감사합니다. 다만, 5년 전에 이루어진 여행에 대한 환불을 요청하신다고 하셨는데, 일반적으로 항공권 환불 정책은 구매 시점부터 일정 기간 내에만 유효합니다.\n\n5년이 지난 경우 환불이 불가능할 가능성이 높습니다. 그러나 정확한 정보를 확인하기 위해 예약 번호나 항공권 정보, 여행 날짜 등을 알려 주시면 정책에 따라 가능한 조치를 확인해 보겠습니다. 어떤 정보로 도와드릴 수 있을지 알려 주시기 바랍니다.')}
----
{'user': HumanMessage(content='예약 번호는 123456이며, 여행 날짜는 2018년 6월 15일입니다. 이 정보를 바탕으로 확인해 주실 수 있을까요?', id='de9cff73-9fa4-47e4-b59a-0560af4eb52e')}
----
{'chat_bot': AIMessage(content='해리슨님, 제공해 주신 예약 번호와 여행 날짜를 바탕으로 확인해 보겠습니다.\n\n### 확인 결과:\n일반적으로 항공사의 환불 정책은 여행 시점에서 일정 기간 내에만 유효하며, 대부분의 항공사는 5년이 지난 후에는 환불을 제공하지 않습니다. \n\n하지만 최종적으로 확실히 확인하기 위해 추가로 예약과 관련된 특별한 약관이나 상황이 있는지 조회해 보겠습니다. 잠시만 기다려 주세요.\n\n(잠시 후)\n\n안타깝게도 확인 결과, 기존 항공권에 대한 환불은 정책상 불가능한 것으로 보입니다. 다만, 이후 여행 